# Utility

- generate directories to help writing batch job commands

In [13]:
import os
from corrLib import readdata

folder = '/home/zhengyang/data/DE/08132021'
l = readdata(folder, 'raw').sort_values(by='Dir')
for num, i in l.iterrows():
    print('python extractImages.py ' + os.path.split(i.Dir)[0] + ' &')

python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h28m24s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h31m03s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h34m17s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h38m55s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h50m58s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_15h56m31s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h02m57s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h05m45s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h08m55s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h12m54s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h15m21s &
python extractImages.py /home/zhengyang/data/DE/08132021/2021-08-13_16h20m05s &
python extractImages.py /home/zhengyang/

In [12]:
i.Dir

'/home/zhengyang/data/DE/08132021/2021-08-13_16h35m25s/RawImage.raw'

In [17]:
# check if folder "images" exist
un_processed = []
for num, i in l.iterrows():
    d = os.path.join(os.path.split(i.Dir)[0], 'images')
    s = os.path.split(os.path.split(i.Dir)[0])[1]
    if os.path.exists(d) == True:
        print(s)
    else:
        un_processed.append(s)
if len(un_processed) != 0:
    print("-------unprocessed------")
    for s in un_processed:
        print(s)

2021-08-13_15h28m24s
2021-08-13_15h31m03s
2021-08-13_15h34m17s
2021-08-13_15h38m55s
2021-08-13_15h50m58s
2021-08-13_15h56m31s
2021-08-13_16h02m57s
2021-08-13_16h05m45s
2021-08-13_16h08m55s
2021-08-13_16h12m54s
2021-08-13_16h15m21s
2021-08-13_16h20m05s
2021-08-13_16h22m52s
2021-08-13_16h26m25s
2021-08-13_16h35m25s


In [4]:
# random stuff
import numpy as np
k = 1.38e-23
T = 300
eta = 1e-3
a = 1e-6
v = 20e-6
D0 = k*T/(6*np.pi*eta*a)
Pe = v*a/D0
Pe

91.0606566257911

In [5]:
91**2 / 9 * 2 +1

1841.2222222222222